In [3]:
import numpy as np
import pandas as pd
import os
import cv2
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import adam
from keras.preprocessing.image import ImageDataGenerator

#Cargamos los datos de trainLabels.csv
#Definiendo las imagenes con la se trabajaran

ubic = 'train'
arch = os.listdir(ubic)
arch.remove("trainLabels.csv") #Eliminamos trainLabels.csv porque no es una imagen 

#Definimos las nuevas dimensiones de las imagenes 

img_filas, img_column = 200, 200

#Creamos dos listas
im_matriz = []
im_Label = []

for file in arch:
    base = os.path.basename('train/' + file)
    fileName = os.path.splitext(base)[0]
    im_Label.append(trainLabels.loc[trainLabels.image==fileName, 'level'].values[0]) #Anadimos los nombre de cada archo y level a la lista im_Label
    im = Image.open('train/' + file)
    img = im.resize((img_filas,img_column))
    gr = img.convert('L') #Convertimos las imagenes a monocromatico
    im_matriz.append(np.array(gr).flatten()) #Anadimos cada imagen en la lista im_matriz (colapsando cada imagen(matriz) en unsa sola dimension)


#Convertimos de lista a array

im_matriz = np.asarray(im_matriz)
im_Label = np.asarray(im_Label)

#Se mezclara los arrays de un amanera consistente

data,Label = shuffle(im_matriz,im_Label, random_state=2)
train_data = [data,Label] #almacena los arrays en train_data en forma de lista

#Preparamos las variables para el entrenamiento

(X, y) = (train_data[0],train_data[1]) #Asignamos de la forma X=data y=label

#Dividimos X, y para el entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

#Redimensionamos las imagenes nuevas
X_train = X_train.reshape(X_train.shape[0], img_column, img_filas, 1)
X_test = X_test.reshape(X_test.shape[0], img_column, img_filas, 1)

#Convertimos las imagenes en el tipo de archivo float32 para poder realizar operaciones 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = X_train/255  
X_test = X_test/255     #Debemos recordar que: "Cada píxel de una imagen almacena la información de su tono o luminosidad, donde el tono negro es el valor 0 y el blanco el valor más alto (normalmente 255 en escala de grises), pero en formato binario".

#Trabajamos con los valores de y_train y_test

clases = 5  #Definimos el numero de clases para la salida

#Convertimos los valores de y_train y_test que son vectores a una matriz de clase binaria
Y_train = np_utils.to_categorical(y_train, clases)
Y_test = np_utils.to_categorical(y_test, clases)

#Ahora se construira la red neuronal convolucional

batch_size = 60  #definimos el numero de muestras que se propagaran en la red neural() 
epocas = 10  #Definimos el numero de epocas para el entrenamiento

filtros = 32 #Definimos el numero de filtros 
nb_pool = 3     #Definimos el tamano de la matriz para reliazar el max pooling
nb_conv = 4     #Tamano del kernel(nucleo) para convolucion


clasificador = Sequential()
clasificador.add(Convolution2D(filtros, kernel_size=(nb_conv, nb_conv), border_mode='valid', input_shape=(img_column, img_filas, 1), activation='relu'))
clasificador.add(Convolution2D(filtros, kernel_size=(nb_conv, nb_conv), activation='relu'))
clasificador.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))

clasificador.add(Convolution2D(2*filtros, kernel_size=(nb_conv, nb_conv), activation='relu'))
clasificador.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))

#clasificador.add(Dropout(0.5)) #El Dropout se usa para ignorar algunas neuronas durante el entrenamiento

clasificador.add(Flatten())
clasificador.add(Dense(128))#Anade 128 neuronas
clasificador.add(Activation('relu'))

clasificador.add(Dense(256))
clasificador.add(Activation('relu'))
clasificador.add(Dropout(0.5))

clasificador.add(Dense(clases)) #Anade en la salida el mismo numero de neuronas que de clases(osea los niveles de retinopatia)
clasificador.add(Activation('softmax'))

clasificador.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

#Se generan lotes de datos de imágenes de tensor con aumento de datos en tiempo real. Los datos se incluirán (en lotes). Estos se usaran para el entrenamiento.

ValidadorDatGenerados = ImageDataGenerator()
GeneradorDatEntr = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,rotation_range=15,zoom_range=0.1 ) #Editamos las imagenes un poco para aumentar el nuemro de datos

#generamos los datos extras con el batch_size especificado anteriormente
train_generator=GeneradorDatEntr.flow(X_train, Y_train, batch_size=batch_size) 
validation_generator=ValidadorDatGenerados.flow(X_test, Y_test,batch_size=batch_size)

#Comenzamos el entrenamiento
clasificador.fit_generator(train_generator, steps_per_epoch=int(len(X_train)/batch_size), epochs=epocas, validation_data=validation_generator, validation_steps=int(len(X_test)/batch_size))

score = clasificador.evaluate(X_test, Y_test, verbose=0)
print(score)


target_dir = 'Introducir la ruta donde se guardara los archivos'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
clasificador.save('modeloFinal.h5')
clasificador.save_weights('pesosFinal.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=(4, 4), input_shape=(200, 200,..., activation="relu", padding="valid")`


Epoch 1/10
20/20 [==============================] - 3s 152ms/step - loss: 1.0390 - acc: 0.7092 - val_loss: 0.9106 - val_acc: 0.7033
Epoch 2/10
20/20 [==============================] - 3s 147ms/step - loss: 0.8776 - acc: 0.7367 - val_loss: 0.9132 - val_acc: 0.7033
Epoch 3/10
20/20 [==============================] - 3s 142ms/step - loss: 0.8844 - acc: 0.7383 - val_loss: 0.8983 - val_acc: 0.7033
Epoch 4/10
20/20 [==============================] - 3s 146ms/step - loss: 0.8812 - acc: 0.7375 - val_loss: 0.9043 - val_acc: 0.7033
Epoch 5/10
20/20 [==============================] - 3s 143ms/step - loss: 0.8803 - acc: 0.7375 - val_loss: 0.9081 - val_acc: 0.7033
Epoch 6/10
20/20 [==============================] - 3s 145ms/step - loss: 0.8704 - acc: 0.7375 - val_loss: 0.9052 - val_acc: 0.7033
Epoch 7/10
20/20 [==============================] - 3s 150ms/step - loss: 0.8847 - acc: 0.7375 - val_loss: 0.9008 - val_acc: 0.7033
Epoch 8/10
20/20 [==============================] - 3s 144ms/step - loss: 0.

In [11]:
import numpy as np
import cv2
import tensorflow as tf
from keras.models import load_model
from PIL import Image
from keras.preprocessing import image

img_filas, img_column = 200, 200
modelo = 'modeloFinal.h5'
pesos_modelo = 'pesosFinal.h5'
clasificador = load_model(modelo)
clasificador.load_weights(pesos_modelo)

def predict(file):

    img = image.load_img(file)
    img = im.resize((img_filas,img_column))
    img = img.convert('L')
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    data = img.astype('float32')
    data = data/255

    array = clasificador.predict(data)
    result = array[0]
    print(result)
    answer = np.argmax(result)

    if answer == 4:
        print('Prediccion:  RD Proliferativa')
    elif answer == 3:
        print('Prediccion:  RD Severo')
    elif answer == 2:
        print('Prediccion:  RD Moderado')
    elif answer == 1:
        print('Prediccion:  RD Leve')
    elif answer == 0:
        print('Prediccion:  No Padece de RD')

    return answer

predict('Introducir la ruta del archivo de prueba')

[0.18784669 0.21716397 0.19344036 0.2246132  0.1769358 ]
Prediccion:  RD Severo


3